<h1>CNN-H1 4-21-2016</h1>

<strong>Abstract</strong>
Implementing the CNN-H1 using NN2 described in the paper: http://arxiv.org/pdf/1509.00244v1.pdf. 

<strong>Implementation</strong>
<ul>
<li>Narrowed the width of faces to focus on the face, cut out background</li>
<li>Validation split of 15%</li>
<li>fc6 layer has a length of 512</li>
<li>Noise with width 15</li>
<li>Number of people is 600</li>
</ul>

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import os
import fnmatch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from scipy.ndimage.filters import gaussian_filter

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

np.random.seed(123456)

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)


In [3]:
data_path = '../data/lfw_cropped'

img_rows_load, img_cols_load = 160, 160
img_rows, img_cols = 160, 120
noise_width = 15
num_people = 600

<h2>Loading Files</h2>

In [4]:
def get_face_to_file_path_dict():
    face_to_file_paths_dict = {}
    
    for root, dirnames, filenames in os.walk(data_path):
        for dirname in dirnames:
            if dirname not in face_to_file_paths_dict:
                face_to_file_paths_dict[dirname] = []
            directory_path = os.path.join(data_path, dirname)
            for filename in os.listdir(directory_path):
                if filename.endswith(".jpg"):
                    face_to_file_paths_dict[dirname].append(os.path.join(directory_path, filename))
                            
    return face_to_file_paths_dict

In [5]:
def get_face_to_file_paths_descending_list(face_to_file_paths_dict):
    return sorted(face_to_file_paths_dict.items(), key=lambda x: len(x[1]), reverse=True)

In [6]:
face_to_file_paths_dict = get_face_to_file_path_dict()

In [7]:
face_to_file_paths_list = get_face_to_file_paths_descending_list(face_to_file_paths_dict)[:num_people]

<h2>Data Pre-Processing</h2>

In [8]:
file_to_facial_feature_points = {}
def get_facial_feature_points(f):
    if f not in file_to_facial_feature_points:
        file_to_facial_feature_points[f] = np.load(f)
    return file_to_facial_feature_points[f]

In [9]:
def image_read(f):
    return rgb2grey(io.imread(f))
    # return resize(rgb2grey(io.imread(f)), (img_rows_load, img_cols_load))

def reflection(image):
    return np.array([list(reversed(row)) for row in image])

def partition(image, top_left, rows, cols):
    return np.array([row[top_left[1]:top_left[1] + cols] for row in image[top_left[0]:top_left[0] + rows]])

In [10]:
def scale_coords(coords, resize_shape, original_shape):
    scale = np.array(resize_shape).astype(float) / np.array(original_shape)
    coords[:,0] = coords[:,0] * scale[1]
    coords[:,1] = coords[:,1] * scale[0]
    return coords

In [11]:
def get_random_noise_image(image, coords, width):
    """
    Apply random gaussian generated values
    and distribute them on gaussian distributed square
    centered on the coordinates passed in for the image
    """
    
    noise = np.zeros((image.shape[0], image.shape[1]))
    for coord in coords:
        # Convert coordinates to rows / columns
        apply_noise_at_point(coord[1], coord[0], noise, width)
    return np.clip(image + noise, 0, 1)

def apply_noise_at_point(x, y, noise, width):
    """
    Generate a block with a single random value placed at the center
    Apply the Gaussian filter with std of 4
    Place it on the noise array at the appropriate coordinates
    
    x represents the rows
    y represents the cols
    """
    
    block = np.zeros((width, width))
    block[width / 2, width / 2] = np.random.normal()
    block = gaussian_filter(block, sigma=4)

    x -= width / 2
    y -= width / 2
    
    x_end = min(noise.shape[0] - x, block.shape[0])
    x_start =  max(0, -x)

    y_end = min(noise.shape[1] - y, block.shape[1])
    y_start = max(0, -y)

    noise[max(0, x):x+block.shape[0], max(0, y):y+block.shape[1]] = block[x_start:x_end,y_start:y_end]

In [12]:
def apply_noise(image, coords):
    return get_random_noise_image(image, coords, noise_width)

In [13]:
images_by_class = [[(image_read(f), get_facial_feature_points(os.path.splitext(f)[0] + '.npy')) 
                    for f in x[1]] for x in face_to_file_paths_list]

In [14]:
# Resize images and scale coordinates
for i in range(len(images_by_class)):
    images_by_class[i] = [(
                  resize(im, (img_rows_load, img_cols_load)),
                  scale_coords(coords, (img_rows_load, img_cols_load), im.shape)
              ) 
              for im, coords in images_by_class[i]]

In [15]:
# Create reflection with reflected coordinates
for i in range(len(images_by_class)):
    for j in range(len(images_by_class[i])):
        im, coords = images_by_class[i][j]
        new_coords = [(im.shape[1] - coord[0], coord[1]) for coord in coords]
        images_by_class[i].append((reflection(im), new_coords))

In [16]:
# Doubling the images and applying random gaussian noise
for i in range(len(images_by_class)):
    images_by_class[i] += images_by_class[i][:]
    images_by_class[i] = [apply_noise(im, coords) for im, coords in images_by_class[i]]

In [17]:
# Partion images to 160x120, similar to the paper
for i in range(len(images_by_class)):
    images_by_class[i] = [partition(im, (0, 20), img_rows, img_cols) for im in images_by_class[i]]

In [18]:
X_train = np.array([image for images in images_by_class for image in images])
y_train = np.array([images[0] for images in enumerate(images_by_class) for image in images[1]])

In [19]:
zipped = np.array(zip(X_train, y_train))
np.random.shuffle(zipped)

In [20]:
X_train = np.array([x[0] for x in zipped])
y_train = np.array([x[1] for x in zipped])

In [21]:
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
Y_train = np_utils.to_categorical(y_train, len(images_by_class))

<h2> Training and Validation</h2>

In [22]:
def NN2(input_shape, nb_classes, nb_fc6):
    model = Sequential()
    
    # Layer 1
    model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=input_shape))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 2
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3))
    model.add(ZeroPadding2D((1,1)))
    model.add(AveragePooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(nb_fc6))
    model.add(Dense(nb_classes, activation='softmax'))
   
    return model

In [23]:
input_shape = (1, img_rows, img_cols)
nb_fc6 = 512

In [24]:
model = NN2(input_shape, num_people, nb_fc6)
model.compile(loss='categorical_crossentropy', optimizer='sgd')
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, 
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 22705 samples, validate on 4007 samples
Epoch 1/10
22705/22705 [==============================] - 708s - loss: 5.8829 - acc: 0.0737 - val_loss: 5.7242 - val_acc: 0.0849
Epoch 2/10
22705/22705 [==============================] - 708s - loss: 5.7676 - acc: 0.0782 - val_loss: 5.7430 - val_acc: 0.0849
Epoch 3/10
22705/22705 [==============================] - 708s - loss: 5.7491 - acc: 0.0782 - val_loss: 5.6880 - val_acc: 0.0849
Epoch 4/10
22705/22705 [==============================] - 708s - loss: 5.6747 - acc: 0.0801 - val_loss: 5.5329 - val_acc: 0.0881
Epoch 5/10
22705/22705 [==============================] - 708s - loss: 5.4540 - acc: 0.0895 - val_loss: 5.1628 - val_acc: 0.1188
Epoch 6/10
22705/22705 [==============================] - 708s - loss: 4.9887 - acc: 0.1342 - val_loss: 4.6655 - val_acc: 0.1789
Epoch 7/10
22705/22705 [==============================] - 708s - loss: 4.0838 - acc: 0.2366 - val_loss: 3.6003 - val_acc: 0.3127
Epoch 8/10
22705/22705 [========================

In [25]:
json_string = model.to_json()
open('models/CNN-H1_fc6-512_people-600_epoch-10.json', 'w').write(json_string)
model.save_weights('models/CNN-H1_fc6-512_people-600_epoch-10.h5', overwrite=True)

In [26]:
model.load_weights('models/CNN-H1_fc6-512_people-600_epoch-10.h5')

In [27]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001))
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10,
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 22705 samples, validate on 4007 samples
Epoch 1/10
22705/22705 [==============================] - 709s - loss: 0.2460 - acc: 0.9307 - val_loss: 0.4772 - val_acc: 0.8944
Epoch 2/10
22705/22705 [==============================] - 709s - loss: 0.1460 - acc: 0.9594 - val_loss: 0.4088 - val_acc: 0.9104
Epoch 3/10
22705/22705 [==============================] - 709s - loss: 0.1077 - acc: 0.9684 - val_loss: 0.3652 - val_acc: 0.9281
Epoch 4/10
22705/22705 [==============================] - 709s - loss: 0.0802 - acc: 0.9778 - val_loss: 0.3355 - val_acc: 0.9364
Epoch 5/10
22705/22705 [==============================] - 709s - loss: 0.0685 - acc: 0.9796 - val_loss: 0.3314 - val_acc: 0.9394
Epoch 6/10
22705/22705 [==============================] - 709s - loss: 0.0545 - acc: 0.9841 - val_loss: 0.3211 - val_acc: 0.9441
Epoch 7/10
22705/22705 [==============================] - 709s - loss: 0.0468 - acc: 0.9864 - val_loss: 0.3051 - val_acc: 0.9496
Epoch 8/10
22705/22705 [========================

In [28]:
json_string = model.to_json()
open('models/CNN-H1_fc6-512_people-600_epoch-20.json', 'w').write(json_string)
model.save_weights('models/CNN-H1_fc6-512_people-600_epoch-20.h5', overwrite=True)

In [29]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001))
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10,
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 22705 samples, validate on 4007 samples
Epoch 1/10
22705/22705 [==============================] - 709s - loss: 0.0339 - acc: 0.9898 - val_loss: 0.3271 - val_acc: 0.9476
Epoch 2/10
22705/22705 [==============================] - 709s - loss: 0.0284 - acc: 0.9922 - val_loss: 0.3107 - val_acc: 0.9493
Epoch 3/10
22705/22705 [==============================] - 709s - loss: 0.0281 - acc: 0.9915 - val_loss: 0.3079 - val_acc: 0.9511
Epoch 4/10
22705/22705 [==============================] - 709s - loss: 0.0245 - acc: 0.9930 - val_loss: 0.3105 - val_acc: 0.9521
Epoch 5/10
22705/22705 [==============================] - 709s - loss: 0.0236 - acc: 0.9932 - val_loss: 0.3105 - val_acc: 0.9538
Epoch 6/10
22705/22705 [==============================] - 709s - loss: 0.0212 - acc: 0.9938 - val_loss: 0.3264 - val_acc: 0.9543
Epoch 7/10
22705/22705 [==============================] - 709s - loss: 0.0194 - acc: 0.9948 - val_loss: 0.3183 - val_acc: 0.9538
Epoch 8/10
22705/22705 [========================

In [30]:
json_string = model.to_json()
open('models/CNN-H1_fc6-512_people-600_epoch-30.json', 'w').write(json_string)
model.save_weights('models/CNN-H1_fc6-512_people-600_epoch-30.h5', overwrite=True)